This post contains some `bash` code-snippet to make my daily work easier.


# Loop through array

In case of looping through an continous list of number:

```{.bash .numberLines .lineAnchors}
#!/bin/bash
for i in {1..100}
do
    # DO SOMETHING
done
```

If the list to loop is not continous, used the following form:

```{.bash .numberLines .lineAnchors}
#!/bin/bash
arr=( 3 4 9 10)
for i in "${arr[@]}"
do
    echo $i       
    #DONE SOMETHING HERE
done

```

Note that `arr` definition is without whitespace.

# Counting number of lines in file

Sometime, after perform a long analysis of many simulations. We want to make sure if the analysis is done automatically, instead of looking into single file one-by-one, we can use `bash` script to do so.

```{.bash .numberLines .lineAnchors}
#!/bin/bash
for i in {1..100}
do
        #echo "$i"
        nl=$(wc -l < $i/G_traj_${i}.dat)
        if [ $nl -eq 400008 ]; then
                echo "traj $i :  $nl --- DONE"
        else
                echo "traj $i :  $nl --- Has not been DONE"
        fi

done

```

In this example, we loop through all simulations (1 to 100) and use command: `wc -l < $i/G_traj_${i}.dat` to count the number of lines in file. we assign the output from this command to `nl` variable by packing previous command by `$(command)`

# Search and replace in file

```{.bash .numberLines .lineAnchors}
#!/bin/bash
cur_dir=`pwd`
for i in {1..100}
do
        if [ -f "/net/people/plgqvuvan/plggligroup/qvuvan/10proj/synthesize/cat3/wt/$i/traj/1/prot_l213_dissociation_final.cor" ]; then
                echo "copy cor and vel file for run $i"
                rm -rf $i && mkdir $i
                cp /net/people/plgqvuvan/plggligroup/qvuvan/10proj/synthesize/cat3/wt/$i/traj/1/prot_l213_dissociation_final.cor $i/
                cp /net/people/plgqvuvan/plggligroup/qvuvan/10proj/synthesize/cat3/wt/$i/traj/1/prot_l213_dissociation_final.vel $i/
                cp template/* $i/
                sed "s/SETINDEX/${i}/g" -i ${i}/job_plgrid_gpu.sh
                sed "s/SETINDEX/${i}/g" -i ${i}/control.cntrl
        else
                echo "simulation $i is not finished"
        fi
done
```

* line 5: check if file `prot_l213_dissociation_final.cor` in specified directory existed. If yes, then run the code block.
* line 11-12: `sed` command used to search and replace pattern `SETINDEX` in file `${i}/job_plgrid_gpu.sh` by value of variable `$i`

# Sync data between local and cluster computer

```{.bash .numberLines .lineAnchors}
#!/bin/bash
rsync -avzhP plgqvuvan@pro.cyfronet.pl:/net/people/plgqvuvan/plggligroup/qvuvan/10proj/post_trans/cat3/stride_by5/combine_trajs/ post_trans/

```


The above command sync the data from cluster pro.cyfronet.pl to local computer.
To exclude files (e.g xtc file), add this: --exclude=*.xtc

# Install package on Roar cluster without Root permission

Roar cluster runs RedHat Linux version 8.9

To install rpm file without root permission: 
`rpm2cpio the_file_name.rpm | cpio -idv`

Example on install ATOM Code editor:

    1. `rpm2cpio atom.x86_64.rpm | cpio -idv`
    This will extract rpm file to folder name ./usr . Move this folder to desired folder, e.g ~/programs:
    2. `mv usr ~/programs/Atom`
    3. Create Desktop link to application:
        Go to ~/.local/share/applications/
        create a desktop file: `Atom.desktop` with content:
        ```
        [Desktop Entry]
        Name=Atom Editor
        Comment=Code Editing. Redefined.
        GenericName=Text Editor
        Exec=/storage/home/qzv5006/programs/Atom/bin/atom %F
        Icon=/storage/home/qzv5006/programs/Atom/share/icons/hicolor/32x32/apps/atom.png
        Type=Application
        StartupNotify=false
        StartupWMClass=Code
        Categories=TextEditor;Development;IDE;
        MimeType=application/x-code-workspace;
        Actions=new-empty-window;
        Keywords=vscode;
        
        [Desktop Action new-empty-window]
        Name=New Empty Window
        Name[de]=Neues leeres Fenster
        Name[es]=Nueva ventana vacÃ­a
        Name[fr]=Nouvelle fenÃªtre vide
        Name[it]=Nuova finestra vuota
        Name[ja]=\u65b0\u3057\u3044\u7a7a\u306e\u30a6\u30a3\u30f3\u30c9\u30a6
        Name[ko]=\uc0c8 \ube48 \ucc3d
        Name[ru]=\u041d\u043e\u0432\u043e\u0435 \u043f\u0443\u0441\u0442\u043e\u0435 \u043e\u043a\u043d\u043e
        Name[zh_CN]=\u65b0\u5efa\u7a7a\u7a97\u53e3
        Name[zh_TW]=\u958b\u65b0\u7a7a\u8996\u7a97
        Exec=/storage/home/qzv5006/programs/Atom/bin/atom --new-window %F
        Icon=/storage/home/qzv5006/programs/Atom/share/icons/hicolor/32x32/apps/atom.png
        ```
    4. Give executable permission for Desktop file: `chmod +x Atom.desktop`